In [4]:
import numpy as np
import random

# Read and store blosum50 scoring matrix from URL
lines = []
with open('blosum50.txt') as f:
    for line in f.readlines():
        # Remove /n and empty list elems from row, convert it into ints
        lines.append([ int(x) for x in list(filter(None, line.rstrip("\n").split(' '))) ])
blosumFifty = np.matrix(lines)

# Lookup table relating row/colum no. to amino acid letter
blosumLookupTable = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

def needlemanWunschAlgorithm(stringOne, stringTwo):
    F = np.empty((len(stringOne)+1, len(stringTwo)+1,))
    d = -8

    # Edge cases are different to middle values in scoring matrix
    for x in range(0, len(stringOne)+1):
        F[x,0] = d*x
    for y in range(0, len(stringTwo)+1):
        F[0,y] = d*y
    for x in range(1, len(stringOne)+1):
        for y in range(1, len(stringTwo)+1):
            match = F[x-1, y-1] + blosumFifty[blosumLookupTable.index(stringOne[x-1]), blosumLookupTable.index(stringTwo[y-1])]
            deletex = F[x-1, y] + d
            deletey = F[x, y-1] + d
            F[x, y] = max(match, deletex, deletey)

    # Print path for algorithm
    path = np.empty((len(stringOne)+1, len(stringTwo)+1,))
    path[:] = 0
    path[len(stringOne), len(stringTwo)] = F[len(stringOne), len(stringTwo)]

    # Backwards dynamic programming algorithm, start from bottom rhs corner and work back to answer
    xPosInMatrix = len(stringOne)
    yPosInMatrix = len(stringTwo)
    resultStringOne = ""
    resultStringTwo = ""
    while (xPosInMatrix > 0 or yPosInMatrix > 0):

        # If there is a diagonal, check if the value came from a "match"
        if xPosInMatrix > 0 and yPosInMatrix>0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix-1, yPosInMatrix-1] + blosumFifty[blosumLookupTable.index(stringOne[xPosInMatrix-1]), blosumLookupTable.index(stringTwo[yPosInMatrix-1])]:
            xPosInMatrix = xPosInMatrix -1
            yPosInMatrix = yPosInMatrix -1
            resultStringOne = stringOne[xPosInMatrix] + resultStringOne
            resultStringTwo = stringTwo[yPosInMatrix] + resultStringTwo

        # If it can go upwards check if the value came from x being deleted and y being replaces
        elif xPosInMatrix > 0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix-1, yPosInMatrix] + d:
            xPosInMatrix = xPosInMatrix -1
            resultStringOne = stringOne[xPosInMatrix] + resultStringOne
            resultStringTwo = "-" + resultStringTwo

        # If it can go across check if the value came from y being deleted and x being replaces
        elif yPosInMatrix > 0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix, yPosInMatrix-1] + d:
            yPosInMatrix = yPosInMatrix -1
            resultStringOne = "-" + resultStringOne
            resultStringTwo = stringTwo[yPosInMatrix] + resultStringTwo

        path[xPosInMatrix, yPosInMatrix] = F[xPosInMatrix, yPosInMatrix]
    print(resultStringOne)
    print(resultStringTwo)
    #print(path)

needlemanWunschAlgorithm("HEAGAWGHEE", "PAWHEAE")
print("This is the same as the result in Lecture 5")
needlemanWunschAlgorithm("PQPTTPVSSFTSGSMLGRTDTALTNTYSAL", "PSPTMEAVEASTASHPHSTSSYFATTYYHL")

HEAGAWGHE-E
--P-AW-HEAE
This is the same as the result in Lecture 5
PQPTTPVSSFTSGSMLGRTDTALTNTYSAL
PSPTMEAVEASTASHPHSTSSYFATTYYHL


In [3]:
def smithWatermanAlgorithm(stringOne, stringTwo):
    F = np.empty((len(stringOne)+1, len(stringTwo)+1,))
    F[:] = 0
    d = -8
    
    # Store the max of all values in the matrix with its x and y co-ordinate
    maxValue = [0, 0, 0]

    # Edge cases are different to middle values in scoring matrix
    for x in range(1, len(stringOne)+1):
        for y in range(1, len(stringTwo)+1):
            match = F[x-1, y-1] + blosumFifty[blosumLookupTable.index(stringOne[x-1]), blosumLookupTable.index(stringTwo[y-1])]
            deletex = F[x-1, y] + d
            deletey = F[x, y-1] + d
            F[x, y] = max(match, deletex, deletey, 0)
            if(F[x, y] > maxValue[0]):
                maxValue = [F[x, y], x, y]

    # Print path for algorithm
    path = np.empty((len(stringOne)+1, len(stringTwo)+1,))
    path[:] = 0
    path[len(stringOne), len(stringTwo)] = F[len(stringOne), len(stringTwo)]

    # Backwards dynamic programming algorithm, start from largest and work back to 0
    xPosInMatrix = maxValue[1]
    yPosInMatrix = maxValue[2]
    resultStringOne = ""
    resultStringTwo = ""
    while (F[xPosInMatrix, yPosInMatrix] != 0):

        # If there is a diagonal, check if the value came from a "match"
        if xPosInMatrix > 0 and yPosInMatrix>0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix-1, yPosInMatrix-1] + blosumFifty[blosumLookupTable.index(stringOne[xPosInMatrix-1]), blosumLookupTable.index(stringTwo[yPosInMatrix-1])]:
            xPosInMatrix = xPosInMatrix -1
            yPosInMatrix = yPosInMatrix -1
            resultStringOne = stringOne[xPosInMatrix] + resultStringOne
            resultStringTwo = stringTwo[yPosInMatrix] + resultStringTwo

        # If it can go upwards check if the value came from x being deleted and y being replaces
        elif xPosInMatrix > 0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix-1, yPosInMatrix] + d:
            xPosInMatrix = xPosInMatrix -1
            resultStringOne = stringOne[xPosInMatrix] + resultStringOne
            resultStringTwo = "-" + resultStringTwo

        # If it can go across check if the value came from y being deleted and x being replaces
        elif yPosInMatrix > 0 and F[xPosInMatrix, yPosInMatrix] == F[xPosInMatrix, yPosInMatrix-1] + d:
            yPosInMatrix = yPosInMatrix -1
            resultStringOne = "-" + resultStringOne
            resultStringTwo = stringTwo[yPosInMatrix] + resultStringTwo

        path[xPosInMatrix, yPosInMatrix] = F[xPosInMatrix, yPosInMatrix]
    print(resultStringOne)
    print(resultStringTwo)
    #print(path)

smithWatermanAlgorithm("HEAGAWGHEE", "PAWHEAE")
print("This is the same as the result in Lecture 6")
smithWatermanAlgorithm("MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY", "TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI")

AWGHE
AW-HE
This is the same as the result in Lecture 6
HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRI
HSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI


In [8]:
def hmmRegion(timesInHMM):
    # Radomly assign starting state
    inATRichState = random.choice([True, False])
    print("Is starting state AT Rich ", inATRichState)
    basesPattern = "["
    for x in range(0, timesInHMM):
        
        # Choose random int for base and transition to calc. next step
        baseProbability = random.randint(0, 10000)
        transitionProbability = random.randint(0, 10000)
        if inATRichState:
            
            # Add Base
            if baseProbability < 2698:
                basesPattern = basesPattern + "a"
            elif 2698 <= baseProbability and baseProbability < 5935:
                basesPattern = basesPattern + "t"
            elif 5935 <= baseProbability and baseProbability < 8015:
                basesPattern = basesPattern + "c"
            else:
                basesPattern = basesPattern + "g"
            
            # If probability is 0.0002 move to CG Rich state
            if transitionProbability > 9998:
                basesPattern = "]["
                inATRichState = False
        else:
            
            # Add Base
            if baseProbability < 2459:
                basesPattern = basesPattern + "a"
            elif 2459 <= baseProbability and baseProbability < 4538:
                basesPattern = basesPattern + "t"
            elif 4538 <= baseProbability and baseProbability < 7016:
                basesPattern = basesPattern + "c"
            else:
                basesPattern = basesPattern + "g"
            
            # If probability is 0.0002 move to CG Rich state
            if transitionProbability > 9997:
                basesPattern = "]["
                inATRichState = False
    return(basesPattern + "]")

print(hmmRegion(1000))

Is starting state AT Rich  False
[tgaatcgtatgcaaaatgtggcttgacggatataccgtattaggctcgcctaacgagaacgcataacaagagcgggagacaaaacaccaaactcgactgttgcgagttgtttgcacctacgttcctttttagcagtcataaaccgccgccctatatcgcactggttccggcgaacctagagctcgaaagcctagggaccaaaataggaggtctggaacggctcctccgtctagggacggttcaaagacttcgcgaggccagcagccataaggctgtgtgtgcgggatctagaggtctaatcaaagggagacatgtggcttgtaccgggttcccactaactcgggcacgggactagctgacggcggtgttacggacctacgcaatgaaccaccttctgcttaaaaaggattgatgtataggaaatgcgagtgctcctctggtaacaagggctgcccggcaggtcgagtggagagaaatttgcgccaaaagaggcttatccaccagtaggaaaagtgcagaagggtcgcatgtatagatgcacttagagctgcgcgtaagaagctgggcctaaagtgttttgagagcgccggtaaggcggaagtggcgtgttgcggaagtcctcttcggcaagcgcactggcggaatccgtgccccgtgcgtggagtcacgtgatggaggtgtgaacttcgcctgccggaggcggcgattcgacaccaggagcgtctagatagcgtggagggttcggataggcatacgtgacccgcgtatgatacaggggcgggtgtcacacgtaatgaagcgcggcccaagggataacggcttagggttccatatgtagcgcgatagtgcggtgtatcacgtatctcccaatacggcgcatcccaatggacatgagaagcgtgcccttcggtacccgaccggcccggtaacaagctaagagcattgaacatggggtgacggatggga

In [ ]:
# Finds the most likely set of states associated with a sequence
# Pass in the states with the dictionary of dictionary
# ASSUMPTION: 2 states
def viterbi(states, sequence):
    # Start in state 0 and store results in matrix 
    v = numpy.zeros((len(states), len(sequence)))
    v[0,0] = 1.0
    v[1,0] = -math.inf
    # Loop through each value in the sequence
    for i in range(1, len(sequence)):
        # Loop through each state- key is a number, values are probabilities
        for j in range(0,2):
            # Choose which state last letter came from based on probability value of last letter
            cameFrom = max(states[0][sequence[i-1]], states[1][sequence[i-1]])
           v[j, i] = states[j][sequence[i]] * max()
        